# Practice SQL with Pandas

---


This notebook is part of a lab done as part of my Immersive Course in Data Science in General Assembly.

In [6]:
# Necessary Libraries:

import pandas as pd
import sqlite3

### Reading data

We'll use the following datasets, available in this repository:

- 'EuroMart-ListOfOrders.csv'
- 'EuroMart-OrderBreakdown.csv'
- 'EuroMart-SalesTargets.csv'

In [7]:
# Reading CSV to Dataframe

orders = pd.read_csv('./datasets/EuroMart-ListOfOrders.csv', encoding = 'utf-8')
OBD =  pd.read_csv('./datasets/EuroMart-OrderBreakdown.csv', encoding = 'utf-8')
sales_targets =  pd.read_csv('./datasets/EuroMart-SalesTargets.csv', encoding = 'utf-8')

In [8]:
# Renaming columns to remove spaces.

orders.columns = ['order_id','order_date','customer_name','city','country','region',
                        'segment','ship_date','ship_mode','state']

OBD.columns = ['order_id','product_name','discount','sales','profit','quantity',
          'category','sub-category']
 
sales_targets.columns = ['month_of_order_date','category','target']

In [9]:
OBD.head()

,order_id,product_name,discount,sales,profit,quantity,category,sub-category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,$45.00,-$26.00,3,Office Supplies,Paper
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,$854.00,$290.00,7,Furniture,Bookcases
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,$140.00,$21.00,3,Office Supplies,Art
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,$27.00,-$22.00,2,Office Supplies,Art
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,$17.00,-$1.00,2,Office Supplies,Storage


In [10]:
# Removing the dollar signs from sales and profit and transforming to float

OBD['sales'] = OBD['sales'].map(lambda x: x.strip('$'))
OBD['sales'] = OBD['sales'].map(lambda x: float(x.replace(',','')))

OBD['profit'] = OBD['profit'].map(lambda x: x.replace('$',''))
OBD['profit'] = OBD['profit'].map(lambda x: float(x.replace(',','')))

In [11]:
OBD.head()

,order_id,product_name,discount,sales,profit,quantity,category,sub-category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,854.0,290.0,7,Furniture,Bookcases
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,21.0,3,Office Supplies,Art
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage


### Creating a SQL Database called 'EuroMart' and save the three dataframes as SQL tables. 

In [12]:
# Now, let's connect to the SQLite database. If no database exists, our command will create one.

db_connection = sqlite3.connect('./datasets/EuroMart.db.sqlite')

These are the basic tools we're going to use to work in between Pandas and SQL:

#### pd.read_sql_table(table_name, con[, schema, ...])
- Reads a SQL database table into a DataFrame.

#### pd.read_sql_query(sql, con[, index_col, ...])
- Reads a SQL query into a DataFrame.

#### pd.read_sql(sql, con[, index_col, ...])
- Reads a SQL query or database table into a DataFrame.
- Adds a convenience wrapper around `read_sql_table()` and `read_sql_query()`.
- Delegates to a specific function, depending on the provided input.

#### DataFrame.to_sql(name, con[, flavor, ...])
- Writes records stored in a DataFrame to a SQL database.

In [13]:
# Let's read our DataFrames as SQL Table

orders.to_sql(name = 'orders', con = db_connection, if_exists = 'replace', index = False)
OBD.to_sql(name = 'order_breakdown', con = db_connection, if_exists = 'replace', index = False)
sales_targets.to_sql(name = 'sales_targets', con = db_connection, if_exists = 'replace', index = False)

### Now we have everyting setted up, let's run some basic queries

#### How many orders has each Customer placed? 

In [14]:
# Getting all customer names and setting them to a pandas object

customers = pd.read_sql('SELECT customer_name FROM orders', con = db_connection)

# Counting unique values in the list

customers['customer_name'].value_counts().head()

Jose Gambino       13
Kayla Tearle       12
Mark Washington    12
Jessica Paramor    11
Patricia Smith     11
Name: customer_name, dtype: int64

In [15]:
# Same thing with pure SQL

pd.read_sql('SELECT customer_name, count(customer_name) as count ' 
             'FROM orders '
             'GROUP BY customer_name '
             'ORDER BY count DESC', 
             con = db_connection).head()

,customer_name,count
0,Jose Gambino,13
1,Kayla Tearle,12
2,Mark Washington,12
3,Aaron Bootman,11
4,Georgina Garner,11


#### Let's query a table of only geographic features from the List of Orders Table.

In [16]:
# City, Country, Region and State are Geographic

pd.read_sql('SELECT DISTINCT city, country, region, state '
             'FROM orders '
             'ORDER BY country', 
             con = db_connection).head()

,city,country,region,state
0,Vienna,Austria,Central,Vienna
1,Linz,Austria,Central,Upper Austria
2,Innsbruck,Austria,Central,Tyrol
3,Graz,Austria,Central,Styria
4,Salzburg,Austria,Central,Salzburg


#### Let's llok for all of the orders that had a negative profit from the Order Breakdown Table.

In [17]:
pd.read_sql('SELECT * ' 
             'FROM order_breakdown '
             'WHERE profit<0', 
             con = db_connection).head()

,order_id,product_name,discount,sales,profit,quantity,category,sub-category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
2,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage
3,BN-2011-3248724,"Ikea Classic Bookcase, Metal",0.6,987.0,-1012.0,6,Furniture,Bookcases
4,BN-2011-3248724,"Binney & Smith Sketch Pad, Blue",0.5,116.0,-56.0,5,Office Supplies,Art


In [18]:
# Alternative way:
# Pretty much identifying any cell in the profit column that has a '-' sign

pd.read_sql('SELECT * '
             'FROM order_breakdown '
             'WHERE profit LIKE "%-%"', 
             con = db_connection).head()

,order_id,product_name,discount,sales,profit,quantity,category,sub-category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
2,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage
3,BN-2011-3248724,"Ikea Classic Bookcase, Metal",0.6,987.0,-1012.0,6,Furniture,Bookcases
4,BN-2011-3248724,"Binney & Smith Sketch Pad, Blue",0.5,116.0,-56.0,5,Office Supplies,Art


#### Let's join two tables to return a table with the Customer Name and Product Name.  

In [20]:
pd.read_sql('SELECT orders.order_id, orders.customer_name, order_breakdown.product_name '
             'FROM orders '
             'LEFT JOIN order_breakdown '
             'ON orders.order_id = order_breakdown.order_id ',
            con = db_connection,index_col='customer_name').head()

,order_id,product_name
customer_name,,
Ruby Patel,BN-2011-7407039,"Enermax Note Cards, Premium"
Summer Hayward,AZ-2011-9050313,"Dania Corner Shelving, Traditional"
Devin Huddleston,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase"
Mary Parker,BN-2011-2819714,"Boston Markers, Easy-Erase"
Mary Parker,BN-2011-2819714,"Eldon Folders, Single Width"


####  How many orders for "Office Supplies" (Category) has Sweden made?

In [21]:
swedish_supplies = pd.read_sql(
            'SELECT orders.order_id, orders.country, order_breakdown.category '            
            'FROM orders '
            'LEFT JOIN order_breakdown '
            'ON orders.order_id = order_breakdown.order_id '
            'WHERE orders.country = "Sweden" AND order_breakdown.category = "Office Supplies"',
            con = db_connection,index_col='country')

len(swedish_supplies)

133

In [22]:
swedish_supplies.head()

,order_id,category
country,,
Sweden,BN-2011-7407039,Office Supplies
Sweden,BN-2011-2807470,Office Supplies
Sweden,BN-2011-7087921,Office Supplies
Sweden,BN-2011-827720,Office Supplies
Sweden,BN-2011-8234232,Office Supplies


#### What was the total sales for products that have been discounted? 

In [24]:
discount_sales = pd.read_sql(
                'SELECT discount, sales '
                'FROM order_breakdown '
                'WHERE discount > 0',
                con = db_connection)

discount_sales['sales'].sum()

1115614.0

In [24]:
# Same query with pure SQL

discount_sales_sum = pd.read_sql(
            'SELECT sum(sales) as total_sales '
            'FROM order_breakdown '
            'WHERE discount > 0',
            con = db_connection,index_col='total_sales')

discount_sales_sum

""
total_sales
1115614.0


#### What is the total quantity of objects sold for each country?

In [25]:
order_counts = pd.read_sql(
                'SELECT order_breakdown.quantity, orders.country '
                'FROM orders '
                'INNER JOIN order_breakdown '
                'ON orders.order_id = order_breakdown.order_id ',
                con = db_connection)

order_counts.groupby('country').sum()

,quantity
country,
Austria,973
Belgium,532
Denmark,204
Finland,201
France,7329
Germany,6179
Ireland,392
Italy,3612
Netherlands,1526


In [26]:
# The same query with pure SQL

order_counts_grouped = pd.read_sql(
                'SELECT sum(order_breakdown.quantity) as total_quantity,'
                'orders.country '
                'FROM orders '
                'INNER JOIN order_breakdown '
                'ON orders.order_id = order_breakdown.order_id '
                'GROUP BY orders.country',
                con = db_connection,index_col='country')

order_counts_grouped

,total_quantity
country,
Austria,973
Belgium,532
Denmark,204
Finland,201
France,7329
Germany,6179
Ireland,392
Italy,3612
Netherlands,1526


#### In what Countries are profits lowest? (lowest 5-10)

In [27]:
# Let's gather country and profits. 

profits = pd.read_sql(
                'SELECT order_breakdown.profit, orders.country '
                'FROM orders '
                'INNER JOIN order_breakdown '
                'ON orders.order_id = order_breakdown.order_id ',
            con = db_connection)

# Groupby country and sum with sort on profits

profits.groupby('country').sum().sort_values(by='profit').reset_index()

,country,profit
0,Netherlands,-37188.0
1,Sweden,-17524.0
2,Portugal,-8704.0
3,Ireland,-6886.0
4,Denmark,-3608.0
5,Finland,3908.0
6,Norway,5167.0
7,Switzerland,7234.0
8,Belgium,9912.0
9,Italy,15802.0


In [28]:
# Same query only using SQL

profits = pd.read_sql(
                'SELECT  orders.country, sum(order_breakdown.profit) as total_profit '
                'FROM orders '
                'INNER JOIN order_breakdown '
                'ON orders.order_id = order_breakdown.order_id '
                'GROUP BY orders.country '
                'ORDER BY total_profit',
            con = db_connection,index_col='country')
profits

,total_profit
country,
Netherlands,-37188.0
Sweden,-17524.0
Portugal,-8704.0
Ireland,-6886.0
Denmark,-3608.0
Finland,3908.0
Norway,5167.0
Switzerland,7234.0
Belgium,9912.0


#### What Countries have the best and worst profit to sales ratios?

In [29]:
# Grabbing Profits, Sales and Countries

spr = pd.read_sql('SELECT order_breakdown.profit, '
                   'order_breakdown.sales, orders.country '
                   'FROM orders '
                   'INNER JOIN order_breakdown '
                   'ON orders.order_id = order_breakdown.order_id ',
            con = db_connection)

# Summing profits and sales by Country

spr2 = spr.groupby('country').sum().sort_values(by='profit')

# creating the ratio column

spr2['ratio'] = spr2['profit']/spr2['sales']

# Sorting by ratio column

spr2.sort_values(by='ratio', ascending = False)

,profit,sales,ratio
country,,,
Switzerland,7234.0,24874.0,0.290826
Austria,21332.0,79382.0,0.268726
Norway,5167.0,20529.0,0.251693
Belgium,9912.0,42320.0,0.234216
United Kingdom,90382.0,420497.0,0.214941
Finland,3908.0,20702.0,0.188774
Spain,47067.0,249402.0,0.188719
Germany,86279.0,488681.0,0.176555
France,70067.0,609683.0,0.114924


In [32]:
# Same using only SQL

spr = pd.read_sql('SELECT orders.country,'
                   'sum(order_breakdown.profit) as total_profit, '
                   'sum(order_breakdown.sales) as total_sales, '
                   '(sum(order_breakdown.profit) / sum(order_breakdown.sales)) as ratio '
                   'FROM orders '
                   'INNER JOIN order_breakdown '
                   'ON orders.order_id = order_breakdown.order_id '
                   'GROUP BY orders.country '
                   'ORDER BY ratio DESC',
            con = db_connection,index_col='country')

spr

,total_profit,total_sales,ratio
country,,,
Switzerland,7234.0,24874.0,0.290826
Austria,21332.0,79382.0,0.268726
Norway,5167.0,20529.0,0.251693
Belgium,9912.0,42320.0,0.234216
United Kingdom,90382.0,420497.0,0.214941
Finland,3908.0,20702.0,0.188774
Spain,47067.0,249402.0,0.188719
Germany,86279.0,488681.0,0.176555
France,70067.0,609683.0,0.114924


#### What city in the Orders table generated the highest net sales? 

In [36]:
pd.read_sql('SELECT orders.city, orders.country, ' 
             'order_breakdown.sales '
             'FROM orders '
             'INNER JOIN order_breakdown '
             'ON orders.order_id = order_breakdown.order_id ',
            con = db_connection).groupby(['city','country']
                ).sum().sort_values(by='sales', ascending = False).head(10)

,,sales
city,country,
London,United Kingdom,69230.0
Berlin,Germany,52555.0
Vienna,Austria,51844.0
Madrid,Spain,44981.0
Paris,France,42245.0
Rome,Italy,28330.0
Barcelona,Spain,27405.0
Hamburg,Germany,23574.0
Marseille,France,21677.0


In [38]:
# Only using SQL

cities = pd.read_sql(
             'SELECT orders.city, orders.country, ' 
             'sum(order_breakdown.sales) as total_sales '
             'FROM orders '
             'INNER JOIN order_breakdown '
             'ON orders.order_id = order_breakdown.order_id '
             'GROUP BY orders.city, orders.country '
             'ORDER BY total_sales DESC '
             ,
            con = db_connection, index_col='city')

cities[:10]

,country,total_sales
city,,
London,United Kingdom,69230.0
Berlin,Germany,52555.0
Vienna,Austria,51844.0
Madrid,Spain,44981.0
Paris,France,42245.0
Rome,Italy,28330.0
Barcelona,Spain,27405.0
Hamburg,Germany,23574.0
Marseille,France,21677.0


#### Let's create a column called 'Shipping Delay' on the 'orders' table, which is the difference in days between 'Order Date' and 'Ship Date'.

In [39]:
from datetime import datetime

In [40]:
# Converting Columns to DateTime Objects from objects
orders['order_date'] = pd.to_datetime(orders['order_date'])
orders['ship_date'] = pd.to_datetime(orders['ship_date'])

In [41]:
# Engineering a feature that counts the difference in Days.
orders['ship_delay'] = (orders['ship_date'])-(orders['order_date'])

In [42]:
orders['ship_delay'] = orders['ship_delay'].astype('timedelta64[D]')

In [43]:
orders.head()

,order_id,order_date,customer_name,city,country,region,segment,ship_date,ship_mode,state,ship_delay
0,BN-2011-7407039,2011-01-01,Ruby Patel,Stockholm,Sweden,North,Home Office,2011-01-05,Economy Plus,Stockholm,4.0
1,AZ-2011-9050313,2011-01-03,Summer Hayward,Southport,United Kingdom,North,Consumer,2011-01-07,Economy,England,4.0
2,AZ-2011-6674300,2011-01-04,Devin Huddleston,Valence,France,Central,Consumer,2011-01-08,Economy,Auvergne-Rhône-Alpes,4.0
3,BN-2011-2819714,2011-01-04,Mary Parker,Birmingham,United Kingdom,North,Corporate,2011-01-09,Economy,England,5.0
4,AZ-2011-617423,2011-01-05,Daniel Burke,Echirolles,France,Central,Home Office,2011-01-07,Priority,Auvergne-Rhône-Alpes,2.0


#### Let's update the Orders table in your Sqlite DB to include the 'Shipping Delay' feature.

In [44]:
# Updating and Replacing the Order datatable

orders.to_sql(name = 'orders', con = db_connection, if_exists = 'replace', index = False)

#### Which Product Category has the highest average 'Shipping Delay'

In [45]:
pd.read_sql(
    'SELECT orders.ship_delay, order_breakdown.category '
    'FROM orders '
    'INNER JOIN order_breakdown '
    'ON orders.order_id = order_breakdown.order_id ',
    con = db_connection).groupby('category').mean()

,ship_delay
category,
Furniture,4.000000
Office Supplies,3.975028
Technology,4.125410


In [46]:
# Only using SQL

pd.read_sql('SELECT avg(orders.ship_delay) as mean_delay, order_breakdown.category '
             'FROM orders '
             'INNER JOIN order_breakdown '
             'ON orders.order_id = order_breakdown.order_id '
             'GROUP BY order_breakdown.category',
             con = db_connection,index_col='category')

,mean_delay
category,
Furniture,4.000000
Office Supplies,3.975028
Technology,4.125410


#### Finally, let's check in what months and Categories were Sales Targets Exceeded?

In [47]:
# Extracting the information from SQL

month_sales = pd.read_sql(
             'SELECT orders.order_date, order_breakdown.sales, order_breakdown.category '
             'FROM orders '
             'INNER JOIN order_breakdown '
             'ON orders.order_id = order_breakdown.order_id ', 
             con = db_connection)

# Convert Orderdate to a datetime object 

month_sales["order_date"] = pd.to_datetime(month_sales["order_date"])

# Creating a column that aggregates dates in 'mon-yy' format

month_sales['month_yr'] = month_sales['order_date'].apply(lambda x: x.strftime('%b-%y'))

In [48]:
# Taking the new date objects and using them to groupby for sum of sales

month_sales = month_sales.groupby(['month_yr','category']).sales.sum().reset_index()

In [49]:
# Pushing this new dataframe that was created with monthly aggregates back to local sql db

month_sales.to_sql(name = 'sales_by_month', con = db_connection, if_exists = 'replace', index = False)

In [50]:
# Extracting information again, join using the newly created table and the sales target tables
# This is a double join in that it is matching values in two columns

targets = pd.read_sql(
            'SELECT sales_targets.month_of_order_date, sales_targets.category, sales_targets.target, sales_by_month.sales '
            'FROM sales_targets '
            'INNER JOIN sales_by_month '
            'ON sales_targets.month_of_order_date = sales_by_month.month_yr AND '
            'sales_targets.category = sales_by_month.category',
            con = db_connection)

In [51]:
# Removing string values and converting target to a float dtype

targets['target'] = targets['target'].map(lambda x: x.replace('$',''))
targets['target'] = (targets['target'].map(lambda x: x.replace(',',''))).astype(float)

In [52]:
# Creating a boolean list that states whether or not sales exceeded the target

exceeded = []
for ind in range(len(targets['target'])):
    if targets['target'][ind] > targets['sales'][ind]:
        exceeded.append(False)
    elif targets['target'][ind] < targets['sales'][ind]:
        exceeded.append(True)

In [53]:
# Appending the list to the dataframe as a column

targets['exceeded'] = exceeded

In [54]:
targets.head()

,month_of_order_date,category,target,sales,exceeded
0,Jan-11,Furniture,10000.0,5477.0,False
1,Feb-11,Furniture,10100.0,13541.0,True
2,Mar-11,Furniture,10300.0,7210.0,False
3,Apr-11,Furniture,10400.0,4115.0,False
4,May-11,Furniture,10500.0,8653.0,False


In [56]:
# Getting those values that exceed

targets[targets['exceeded'] == True].head(10)

,month_of_order_date,category,target,sales,exceeded
1,Feb-11,Furniture,10100.0,13541.0,True
5,Jun-11,Furniture,10600.0,14737.0,True
8,Sep-11,Furniture,11000.0,13763.0,True
10,Nov-11,Furniture,11300.0,15194.0,True
11,Dec-11,Furniture,11400.0,23611.0,True
17,Jun-12,Furniture,12100.0,21661.0,True
19,Aug-12,Furniture,12400.0,21300.0,True
20,Sep-12,Furniture,12500.0,20161.0,True
21,Oct-12,Furniture,12600.0,14923.0,True
22,Nov-12,Furniture,12800.0,15100.0,True


In [57]:
# Getting those values that did not exceed expectations.

targets[targets['exceeded'] == False].head(10)

,month_of_order_date,category,target,sales,exceeded
0,Jan-11,Furniture,10000.0,5477.0,False
2,Mar-11,Furniture,10300.0,7210.0,False
3,Apr-11,Furniture,10400.0,4115.0,False
4,May-11,Furniture,10500.0,8653.0,False
6,Jul-11,Furniture,10800.0,2282.0,False
7,Aug-11,Furniture,10900.0,10606.0,False
9,Oct-11,Furniture,11100.0,4084.0,False
12,Jan-12,Furniture,11500.0,5525.0,False
13,Feb-12,Furniture,11600.0,5820.0,False
14,Mar-12,Furniture,11800.0,9496.0,False
